# Imports

In [ ]:
from pathlib import Path
import pandas as pd
from pandas.plotting import scatter_matrix
import tarfile
import urllib.request
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler,  StandardScaler, FunctionTransformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_selector, make_column_transformer
from sklearn.cluster import KMeans
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics.pairwise import rbf_kernel
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error, mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score, GridSearchCV, RandomizedSearchCV
from scipy import stats
import joblib

sklearn.set_config(display='diagram')


# Lectura de los datos

In [ ]:
def load_housing_data():
    tarball_path = Path("./datasets/housing.tgz")
    if not tarball_path.is_file():
        Path("datasets").mkdir(parents=True, exist_ok=True)
        url = "https://github.com/ageron/data/raw/main/housing.tgz"
        urllib.request.urlretrieve(url, tarball_path)
        with tarfile.open(tarball_path) as housing_tarball:
            housing_tarball.extractall(path="datasets")
    return pd.read_csv(Path("./datasets/housing/housing.csv"))

In [ ]:
housing = load_housing_data()

In [ ]:
# Vistazo rapido a la estructura de datos
housing.head()

# Exploracion Inicial

Cada Registro es un distrito

In [ ]:
# Informacion general de los datos
housing.info()

In [ ]:
# Feature Categoricas
housing["ocean_proximity"].value_counts()

Estos Datos son por distritos y no por casas

In [ ]:
# Feature Numerica - Se ignora los valores nulos
housing.describe()

In [ ]:
# Histograma de las features
housing.hist(bins=100, figsize=(20,15))
plt.show()

# Crear conjuto de pruebas y entrenamiento

In [ ]:
# Separacion de conjuntos sin estratificacion
# X_train, X_test = train_test_split(housing, test_size=0.2, random_state=42)

Para que una separacion de conjuntos sea mas efectiva hacemos una separacion estratificada segun una variable importante en este caso seleccionamos el `median_income` pero tenemos que transformarla a una variable categorica

In [ ]:
housing['median_income'].describe()

In [ ]:
# Transformacion de la feature "median_income" a una feature categorica
housing["income_cat"] = pd.cut(housing["median_income"],
                               bins=[0., 1.5, 3.0, 4.5, 6., np.inf],
                               labels=[1, 2, 3, 4, 5])

In [ ]:
# Evaluamos la distribucion
housing["income_cat"].value_counts().sort_index().plot.bar(rot=0,grid=True)
plt.xlabel
plt.ylabel("Number of Districts")
plt.show()

In [ ]:
strat_train_set, strat_test_set = train_test_split(housing, test_size=0.2, random_state=42, stratify=housing["income_cat"])
# Luego de usar esta columna la eliminamos
for set_ in (strat_train_set, strat_test_set):
    set_.drop("income_cat", axis=1, inplace=True)

In [ ]:
# Hacemos una copia del dataset de entrenamiento para analizarlo
housing = strat_train_set.copy()

# EDA y Visualizacion de datos

## Datos Geograficos

In [ ]:
# Visualizar Datos Geograficos
housing.plot(kind="scatter", x="longitude", y="latitude", alpha=0.2, grid=True)
plt.show()

In [ ]:
housing.plot(kind="scatter", x="longitude", y="latitude", grid=True, s=housing["population"]/100, label="population",
             c="median_house_value", cmap="jet", colorbar=True, legend=True, sharex=False, figsize=(10,7))
plt.show()

## Correlaciones

In [ ]:
# Exclude non-numeric columns for correlation matrix
corr_matrix = housing.drop("ocean_proximity", axis=1).corr()
corr_matrix["median_house_value"].sort_values(ascending=False)

In [ ]:
attributes = ['median_house_value', 'median_income', 'total_rooms', 'housing_median_age']
scatter_matrix(housing[attributes], figsize=(12, 8))
plt.show()

In [ ]:
housing.plot(kind="scatter", x="median_income", y="median_house_value", alpha=0.1, grid=True)
plt.show()

# Experimentar con combinaciones de atributos

In [ ]:
# Creamos nuesvos atributos
housing["rooms_per_house"] = housing["total_rooms"] / housing["households"]
housing["bedrooms_ratio"] = housing["total_bedrooms"] / housing["total_rooms"]
housing["people_per_house"] = housing["population"] / housing["households"]

In [ ]:
corr_matrix = housing.drop("ocean_proximity", axis=1).corr()
corr_matrix["median_house_value"].sort_values(ascending=False)

In [ ]:
housing = strat_train_set.drop("median_house_value", axis=1) # Guardamos los datos de entrenamiento sin la variable a predecir
housing_labels = strat_train_set["median_house_value"].copy() # Guardamos la variable a predecir 

# Limpiar Datos

## Features Numericas

In [ ]:
# Vamos a limpiar los datos nulos con un imputer
impute = SimpleImputer(strategy="median") # Creamos un imputer con la estrategia de reemplazo de valores nulos por la mediana
housing_num = housing.select_dtypes(include=[np.number]) # Seleccionamos solo las columnas numericas

impute.fit(housing_num) # Ajustamos el imputer a los datos
X = impute.transform(housing_num) # Transformamos los datos, Genera una salida de tipo numpy array
housing_tr = pd.DataFrame(X, columns=housing_num.columns, index=housing_num.index) # Convertimos el array a un dataframe de pandas

## Features Categoricas

In [ ]:
# Atributos de texto o categoticos
housing_cat = housing[["ocean_proximity"]]
housing_cat.value_counts()

In [ ]:
cat_encoder = OneHotEncoder(sparse_output=False) # Creamos un encoder para las variables categoricas
housing_cat_1hot = cat_encoder.fit_transform(housing_cat) # Transformamos las variables categoricas a numericas
type(housing_cat_1hot) # Salida de tipo scipy sparse matrix

In [ ]:
cat_encoder.categories_ # Categorias de las variables categoricas

# Escalado de caracteristicas y Transformacion

In [ ]:
# Normalizacion de los datos
min_max_scaler = MinMaxScaler() # Creamos un escalador para normalizar los datos
housing_num_min_max_scaled = min_max_scaler.fit_transform(housing_num) # Normalizamos los datos

In [ ]:
std_scaler = StandardScaler() # Creamos un escalador para estandarizar los datos
housing_num_std_scaled = std_scaler.fit_transform(housing_num) # Estandarizamos los datos

In [ ]:
log_transformer = FunctionTransformer(np.log, inverse_func=np.exp) # Creamos un transformador para aplicar logaritmo a los datos, El argumento inverse_func es opcional
log_pop = log_transformer.transform(housing[["population"]]) # Aplicamos el logaritmo a la columna "population"

In [ ]:
# Transformador personalizado 
class ClusterSimilarity(BaseEstimator, TransformerMixin):
    """
    A custom transformer that computes the similarity of each sample to the cluster centers
    using the Radial Basis Function (RBF) kernel.

    Parameters:
    n_clusters (int): The number of clusters to form as well as the number of centroids to generate.
    gamma (float): The gamma parameter for the RBF kernel.
    random_state (int, RandomState instance or None): Determines random number generation for centroid initialization.
    """
    def __init__(self, n_clusters=10, gamma=1.0, random_state=None):
        self.n_clusters = n_clusters
        self.gamma = gamma
        self.random_state = random_state

    def fit(self, X, y=None, sample_weight=None):
        """
        Compute k-means clustering.

        Parameters:
        X (array-like or sparse matrix, shape=(n_samples, n_features)): Training instances to cluster.
        y (ignored): Not used, present here for API consistency by convention.
        sample_weight (array-like, shape (n_samples,), optional): The weights for each observation in X.

        Returns:
        self: Fitted estimator.
        """
        self.kmeans_ = KMeans(n_clusters=self.n_clusters, random_state=self.random_state)
        self.kmeans_.fit(X, sample_weight=sample_weight)
        return self
    
    def transform(self, X):
        """
        Transform X to a cluster-distance space.

        Parameters:
        X (array-like or sparse matrix, shape=(n_samples, n_features)): New data to transform.

        Returns:
        array, shape (n_samples, n_clusters): X transformed in the new space.
        """
        return rbf_kernel(X, self.kmeans_.cluster_centers_, gamma=self.gamma)
    
    def get_feature_names_out(self, names=None):
        """
        Get output feature names for transformation.

        Parameters:
        names (array-like of str or None): Input feature names.

        Returns:
        list of str: Transformed feature names.
        """
        return [f"Cluster {i} similarity" for i in range(self.n_clusters)]

# Pipeline

In [ ]:
num_pipeline = Pipeline([
    ("impute", SimpleImputer(strategy="median")), # Imputamos los valores nulos
    ("standardize", StandardScaler()) # Estandarizamos los datos
])

In [ ]:
cat_pipeline = Pipeline([
    ("Impute", SimpleImputer(strategy="most_frequent")),
    ("Encoder", OneHotEncoder(handle_unknown="ignore", sparse_output=False))
])

In [ ]:
housing_num_prepared = num_pipeline.fit_transform(housing_num) # Aplicamos el pipeline a los datos numericos
housing_num_prepared[:2].round(2) # Mostramos los primeros 2 registros

In [ ]:
df_housing_num_prepared = pd.DataFrame(data=housing_num_prepared,
                                       columns=num_pipeline.get_feature_names_out(),
                                       index=housing_num.index)
df_housing_num_prepared.head()  

a

In [ ]:
num_attribs = list(housing_num.columns) # Lista de atributos numericos
cat_attribs = list(housing_cat.columns) # Lista de atributos categoricos

In [ ]:
# Preprocesamiento de los datos con ColumnTransformer
preprocessing_name = ColumnTransformer([
    ("num", num_pipeline, num_attribs),
    ("cat", cat_pipeline, cat_attribs),
])
preprocessing_name

In [ ]:
preprocessing = make_column_transformer(
    (num_pipeline, make_column_selector(dtype_include=np.number)),
    (cat_pipeline, make_column_selector(dtype_include=object))
    )
preprocessing

In [ ]:
housing_prepared = preprocessing.fit_transform(housing) # Aplicamos el preprocesamiento a los datos
print(type(housing_prepared)) # Salida de tipo numpy array



### Transformacion Final

In [ ]:
def column_ratio(X):
    """
    Calcula la razón entre la primera y la segunda columna de un array numpy.

    Parámetros:
    X (numpy.ndarray): Un array numpy de dos o más columnas.

    Retorna:
    numpy.ndarray: Un array numpy con la razón entre la primera y la segunda columna.
    """
    return X[:, [0]] / X[:, [1]]


def ratio_name(function_transformer, features_names_in):
    """
    Genera el nombre de la nueva característica creada por el transformador de función.

    Parámetros:
    function_transformer (FunctionTransformer): El transformador de función que se está utilizando.
    features_names_in (list): Lista de nombres de las características de entrada.

    Retorna:
    list: Una lista con el nombre de la nueva característica.
    """
    return ["ratio"]


def ratio_pipeline():
    """
    Crea un pipeline para calcular la razón entre la primera y la segunda columna,
    imputar valores nulos con la mediana y estandarizar los datos.

    Retorna:
    sklearn.pipeline.Pipeline: Un pipeline de scikit-learn.
    """
    return make_pipeline(
        SimpleImputer(strategy="median"),
        FunctionTransformer(column_ratio, feature_names_out=ratio_name),
        StandardScaler()
    )



In [ ]:
log_pipeline = make_pipeline(
    SimpleImputer(strategy="median"),
    FunctionTransformer(np.log, feature_names_out="one-to-one"),
    StandardScaler()
)

cluster_simil = ClusterSimilarity(n_clusters=10, gamma=1.0, random_state=42)

default_num_pipeline = make_pipeline(
    SimpleImputer(strategy="median"),
    StandardScaler()
)

In [ ]:
preprocessing_final = ColumnTransformer([
    ("bedrooms", ratio_pipeline(), ["total_bedrooms", "total_rooms"]),
    ("rooms_per_house", ratio_pipeline(), ["total_rooms", "households"]),
    ("people_per_house", ratio_pipeline(), ["population", "households"]),
    ("log", log_pipeline, ["total_bedrooms", "total_rooms", "population", "households", "median_income"]),
    ("geo", cluster_simil, ["latitude", "longitude"]),
    ("cat", cat_pipeline, make_column_selector(dtype_include=object)),
],
remainder=default_num_pipeline,
verbose=True
)
preprocessing_final

In [ ]:
housing_prepared_final = preprocessing_final.fit_transform(housing)

In [ ]:
housing_prepared_final.shape

In [ ]:
preprocessing_final.get_feature_names_out()

In [ ]:
type(housing_prepared_final)


# Entrenar Modelos

## Linear Regression

In [ ]:
lin_reg = make_pipeline(preprocessing_final, LinearRegression())
lin_reg

In [ ]:
lin_reg.fit(housing, housing_labels)

In [ ]:
housing_predictions = lin_reg.predict(housing)
housing_predictions[:5].round(-2)


In [ ]:
lin_rmse = root_mean_squared_error(housing_labels, housing_predictions)
lin_rmse

## Decision Tree Regressor

In [ ]:
tree_reg = make_pipeline(preprocessing_final, DecisionTreeRegressor(random_state=42))
tree_reg

In [ ]:
tree_reg.fit(housing, housing_labels)

In [ ]:
housing_predictions = tree_reg.predict(housing)
housing_predictions[:5]

In [ ]:
tree_rmse = root_mean_squared_error(housing_labels, housing_predictions)
tree_rmse

## Cross Validation

In [ ]:
tree_rmses = cross_val_score(estimator=tree_reg, X=housing, y=housing_labels, scoring="neg_root_mean_squared_error", cv=10)
# Since sklearn's cross_val_score returns negative values for error metrics,
# we need to negate them to get the actual RMSE values
pd.Series(-tree_rmses).describe()

## Random Forest

In [ ]:
forest_reg = make_pipeline(preprocessing_final, RandomForestRegressor(random_state=42))

In [ ]:
forest_rmses = -cross_val_score(estimator=forest_reg, X=housing, y=housing_labels,
                                scoring="neg_root_mean_squared_error", cv=10)

In [ ]:
pd.Series(forest_rmses).describe()

## Busqueda exhaustiva

In [ ]:
full_pipeline = Pipeline([ # Creamos un pipeline con los pasos de preprocesamiento y el modelo
    ("preprocessing", preprocessing_final),
    ("random_forest", RandomForestRegressor(random_state=42))
])

param_grid = [
    {
        'preprocessing__geo__n_clusters': [5, 8, 10], # Number of clusters on the geo transformer on the preprocessing step
        'random_forest__max_features': [4, 5, 9] # Number of features to consider when looking for the best split
    },
    {
        'preprocessing__geo__n_clusters': [10, 15], # number of clusters on the geo transformer on the preprocessing step
        'random_forest__max_features': [6, 8, 10] # number of features to consider when looking for the best split
    }
]

grid_search = GridSearchCV(estimator=full_pipeline, param_grid=param_grid,
                           scoring="neg_root_mean_squared_error", cv=3, verbose=2)

In [ ]:
grid_search.fit(housing, housing_labels)

In [ ]:
grid_search.best_params_

In [ ]:
grid_search.best_score_

In [ ]:
cv_res = pd.DataFrame(grid_search.cv_results_)
cv_res.sort_values(by="mean_test_score", ascending=False, inplace=True)
cv_res

## Busqueda Random

In [ ]:
param_distribs = {
    'preprocessing__geo__n_clusters': [np.random.randint(low=3, high=50)],
    'random_forest__max_features': [np.random.randint(low=2, high=20)]
}

rnd_search = RandomizedSearchCV(
    full_pipeline, param_distributions=param_distribs, n_iter=50, cv=3,
    scoring='neg_root_mean_squared_error', random_state=42, verbose=2
)

In [ ]:
rnd_search.fit(housing, housing_labels)

In [ ]:
rnd_search.best_params_

In [ ]:
rnd_search.best_score_

In [ ]:
final_model = rnd_search.best_estimator_ # Guardamos el mejor modelo

In [ ]:
features_importances = final_model["random_forest"].feature_importances_ # Obtenemos las importancias de las features
features_importances.round(2) # Mostramos las importancias de las features

In [ ]:
sorted(zip(features_importances, final_model["preprocessing"].get_feature_names_out()), reverse=True) # Mostramos las importancias de las features ordenadas de mayor a menor

# Evaluar modelo

## Evaluar con el conjunto de prueba

In [ ]:
X_test = strat_test_set.drop("median_house_value", axis=1) # Guardamos los datos de test sin la variable a predecir
y_test = strat_test_set["median_house_value"].copy() # Guardamos la variable a predecir

In [ ]:
final_prediction = final_model

In [ ]:
final_rmse = root_mean_squared_error(y_test, final_prediction) # Calculamos el RMSE de la prediccion
print(final_rmse)

In [ ]:
confidence = 0.95
squared_erros = (final_prediction - y_test) ** 2 # Calculamos el error cuadratico
np.sqrt(stats.t.interval(
    confidence=confidence,
    df=len(squared_erros)-1,
    loc=np.mean(squared_erros),
    scale=stats.sem(squared_erros)
))

# Lanzar Modelo

In [ ]:
joblib.dump(final_model, "my_california_housing_model.pkl") # Guardamos el modelo en un archivo

In [ ]:
final_model_realoaded = joblib.load("my_california_housing_model.pkl") # Cargamos el modelo desde el archivo

In [ ]:
new_data = X_test.iloc[:5] # Cargamos los primeros 5 registros del conjunto de test
predictions = final_model_realoaded.predict(new_data) # Hacemos la prediccion con el modelo recargado
predictions